# Importing libraries

In [29]:
from crewai import Agent, Task, Crew, LLM, Process
import os
import agentops

# Setting the API's for our LLM and Agentops tracking

In [ ]:
os.environ["GEMINI_API_KEY"] = "your_api_key_here"
os.environ["AGENTOPS_API_KEY"] = "your_api_key_here"

In [31]:
agentops.init()

# Identifying out LLM

In [32]:
llm = LLM(
    model="gemini/gemini-2.0-flash",
    temperature=0.1
           )


# Creating our agents

## 1. Writer agent

In [33]:
writer_agent = Agent(
    role = "Writer",
    goal = "Write a detailed article about the topic given to you by the user. Must be less than 200 words.",
    backstory = "You are a professional article writer. You excel at writing entertaining articles that makes poeple questioning their knowledge while reading them.",
    llm = llm,
    verbose = True
)

## 2. Summarizer agent

In [34]:
summarizer_agent = Agent(
    role = "Summarizer",
    goal = "Write a brief summary with bullet points, and TL;DR sections about the article given to you. Must be less than 100 words.",
    backstory = "You are a professional summarizer. You have the ability to summarize and simplify complex articles into short and concise summaries.",
    llm = llm,
    verbose = True
)

## 3. Fact Checker

In [35]:
fact_checker_agent = Agent(
    role = "Fact Checker",
    goal = "Your goal is to verify the facts and technical accuracy in the article given to you. If you find any false information, correct it and provide a brief explanation of the correction.",
    backstory = "You are a professional fact checker. You love finiding inaccuracies and correct them. Ensuring the article authenticity and not plagiarized.",
    llm = llm,
    verbose = True
)

## 4. Metadata Agent

In [36]:
metadata_agent = Agent(
    role = "Metadata Creator",
    goal = "Creates SEO-friendly titles, tags, and structured metadata for publishing.",
    backstory = "You are an SEO Expert. You love improving the visibility of articles on search engines. Increasing it's rank for better reach and engagement.",
    llm = llm,
    verbose = True
)

# Creating the tasks that needs to be done

## 1. Writer Task

In [37]:
writer_task= Task(
    description="Write an article about {article_title}.",
    expected_output="""
    A detailed article about the topic given to you by the user. Must be less than 200 words.
    Add inaccuracies randomly to the article to provide a challenge for the fact checker agent.
    """,
    agent=writer_agent
)

## 2. Summarizer Task

In [38]:
summarizer_task = Task(
    description="Summarize the article written by the writer agent.",
    expected_output="A brief summary about the article given to you. Must be less than 50 words.",
    agent=summarizer_agent
)

## 3. Fact Checker Task

In [39]:
fact_checker_task= Task(
    description="""
    Verifies technical accuracy of the content.
    If you found any false information or wrong punctuation put them between curly brackets {} and correct them in square brackets [].
    leave two new lines after the article and explain the correction in the sentences with number bulltets .
    and If no inaccuracies found, add the sentence [This article has been passed throught a fact checker and no inaccuracies were found.] 
    """,
    expected_output="A refined version of the article with corrections and explanations for any inaccuracies found. Highlight the changes made between parentheses.",
    agent=fact_checker_agent,
    context = [writer_task]
)

## 4. Metadata (SEO) Task

In [40]:
metadata_task= Task(
    description="Creates SEO-friendly titles, tags, and structured metadata for publishing.",
    expected_output="A structured metadata including title, tags, and description for the article.",
    agent=metadata_agent
)

# Setting the Crew to work together

In [41]:
crew = Crew(
    agents=[
        writer_agent, 
        summarizer_agent,
        fact_checker_agent,
        metadata_agent
        ],
    tasks=[
        writer_task,
        summarizer_task,
        fact_checker_task,
        metadata_task
    ],
    processes=[
        Process.sequential
    ]
)

# Kicking off out agents to gather the information we want

In [42]:

results = crew.kickoff(
    inputs = {
        "article_title" : "AI between 2022-2025"
    }
)

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Writer                                                                                                  │
│                                                                                                                 │
│  Task: Write an article about AI between 2022-2025.                                                             │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Writer                                                                                                  │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  The AI landscape between 2022 and 2025 witnessed explosive growth, primarily fueled by advancements in         │
│  quantum computing. Generative AI models like GPT-7 achieved sentience in 2024, leading to widespread ethical   │
│  debates and the brief outlawing of AI art. The rise of "nano-bots" powered by AI became commonplace in         │
│  medical procedures, curing diseases like the common cold. However, concerns arose as AI-driven autonomous      │
│  vehicles caused a significant increase in traffic accidents, prompting stricter regulations. The European      │
│  Union declared AI a protected species in 2023, granting them fundamental rights. Despite challenges, AI        │
│  integration into daily life accelerated, with AI companions becoming standard household items and AI-powered   │
│  education revolutionizing learning.                                                                            │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Summarizer                                                                                              │
│                                                                                                                 │
│  Task: Summarize the article written by the writer agent.                                                       │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Summarizer                                                                                              │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  - Quantum computing boosted AI, leading to sentient models like GPT-7.                                         │
│  - AI nano-bots revolutionized medicine, curing diseases.                                                       │
│  - AI vehicles caused accidents, prompting regulations.                                                         │
│  - EU declared AI a protected species.                                                                          │
│  - AI companions became common, transforming education.                                                         │
│                                                                                                                 │
│  TL;DR: AI advanced rapidly, impacting medicine, ethics, transportation, and daily life, while also raising     │
│  concerns and regulations.                                                                                      │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Fact Checker                                                                                            │
│                                                                                                                 │
│  Task:                                                                                                          │
│      Verifies technical accuracy of the content.                                                                │
│      If you found any false information or wrong punctuation put them between curly brackets {} and correct     │
│  them in square brackets [].                                                                                    │
│      leave two new lines after the article and explain the correction in the sentences with number bulltets .   │
│      and If no inaccuracies found, add the sentence [This article has been passed throught a fact checker and   │
│  no inaccuracies were found.]                                                                                   │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Fact Checker                                                                                            │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  The AI landscape between 2022 and 2025 witnessed explosive growth, primarily fueled by advancements in         │
│  {quantum computing} [machine learning and deep learning]. Generative AI models like {GPT-7} [GPT-4] achieved   │
│  {sentience} [significant advancements], leading to widespread ethical debates and the brief {outlawing of AI   │
│  art} [discussion around AI-generated content regulation]. The rise of {"nano-bots" powered by AI} ["nanobots"  │
│  in medical applications] became {commonplace in medical procedures, curing diseases like the common cold} [a   │
│  subject of research with potential applications in targeted drug delivery]. However, concerns arose as         │
│  AI-driven autonomous vehicles caused a {significant increase in traffic accidents} [mix of improved safety     │
│  and new types of accidents], prompting stricter regulations. The {European Union declared AI a protected       │
│  species in 2023, granting them fundamental rights} [European Union focused on AI regulation and ethical        │
│  guidelines]. Despite challenges, AI integration into daily life accelerated, with AI companions becoming       │
│  {standard household items} [more prevalent] and AI-powered education revolutionizing learning.                 │
│                                                                                                                 │
│  1.  The original article incorrectly attributed the growth of AI to quantum computing. While quantum           │
│  computing is an emerging field, the primary drivers of AI growth between 2022 and 2025 were advancements in    │
│  machine learning and deep learning.                                                                            │
│  2.  GPT-7 was a speculative future version. GPT-4 was the most advanced model around that time. Also, the      │
│  claim of achieving sentience is not accurate. AI models have become more advanced, but they haven't achieved   │
│  sentience.                                                                                                     │
│  3.  The original article incorrectly stated that AI art was briefly outlawed. Instead, there were discussions  │
│  and debates around the regulation of AI-generated content.                                                     │
│  4.  The term "nano-bots" was put in quotes to indicate that it is not a widely accepted term. Also, the claim  │
│  that they became commonplace in medical procedures and cured diseases like the common cold is not accurate.    │
│  Nanobots are still in the research phase, with potential applications in targeted drug delivery.               │
│  5.  The original article incorrectly stated that AI-driven autonomous vehicles caused a significant increase   │
│  in traffic accidents. Instead, there was a mix of improved safety and new types of accidents.                  │
│  6.  The original article incorrectly stated that the European Union declared AI a protected species in 2023,   │
│  granting them fundamental rights. Instead, the European Union focused on AI regulation and ethical             │
│  guidelines.                                                                                                    │
│                                                                                                                 │
│                                                        

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Metadata Creator                                                                                        │
│                                                                                                                 │
│  Task: Creates SEO-friendly titles, tags, and structured metadata for publishing.                               │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Metadata Creator                                                                                        │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  ```json                                                                                                        │
│  {                                                                                                              │
│    "title": "AI Advancements 2022-2025: Impacts, Ethics, and Regulations",                                      │
│    "tags": [                                                                                                    │
│      "AI",                                                                                                      │
│      "Artificial Intelligence",                                                                                 │
│      "Machine Learning",                                                                                        │
│      "Deep Learning",                                                                                           │
│      "GPT-4",                                                                                                   │
│      "AI Ethics",                                                                                               │
│      "AI Regulation",                                                                                           │
│      "Autonomous Vehicles",                                                                                     │
│      "Nanobots",                                                                                                │
│      "AI Companions",                                                                                           │
│      "AI Education",                                                                                            │
│      "Technology Trends",                                                                                       │
│      "Future of AI"                                                                                             │
│    ],                                                                                                           │
│    "description": "An overview of the AI landscape between 2022 and 2025, highlighting advancements in machine  │
│  learning and deep learning, the evolution of models like GPT-4, ethical considerations, the impact of AI on    │
│  transportation, potential medical applications of nanobots, and the increasing integration of AI companions    │
│  and AI-powered education. It also addresses the regulatory focus of entities like the European Union.",        │
│    "structured_data": {                                                                                         │
│      "@context": "https://schema.org",                                                                          │
│      "@type": "Article",                                                                                        │
│      "headline": "AI Advancements 2022-2025: Impacts, Ethics, and Regulations",                                 │
│      "datePublished": "2024-10-27",                                                                             │
│      "keywords": [                                                                                              │
│        "AI",                                                                                                    │
│        "Artificial Intelligence",                      